#### **0. Libraries**

In [2]:
# imporing required libraries   
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm
import yaml
import os
from scipy import linalg
from PIL import Image


c:\Users\Admin\anaconda3\envs\ddpm-pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **1. Utility functions**

In [3]:
## Custom dataset ##
class CustomDataset(Dataset):
    def __init__(self, target_dir, transform=None):
        self.paths = [ os.path.join(target_dir, x) for x in os.listdir(target_dir) if x.endswith('.npy')]
        # load all the paths of numpy images from the target directory    
        self.transform = transform
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        img_path = self.paths[index]
        img = np.load(img_path) # (1,150,150)     
        img = torch.tensor(img, dtype=torch.float32)

        if self.transform:
            img = self.transform(img)
            
        return img 

In [4]:
## FID score computation ##

def get_activation(dataloader, model, preprocess, # Preprocessing Transform for InceptionV3
                   device = 'cpu'
                  ):
    """
    Given Dataloader and Model, Generate N X 2048
    Dimensional activation map for N data points
    in dataloader.
    """
    # Set model to evaluation Mode
    model.to(device)
    model.eval()
    # Save activations
    pred_arr = np.zeros((len(dataloader.dataset), 2048))
    batch_size = dataloader.batch_size
    
    with torch.no_grad():
        for i, batch in tqdm(enumerate(dataloader)):
            # Transform the Batch according to Inceptionv3 specification
            batch = torch.stack([preprocess(img) for img in batch]).to(device)
            pred = model(batch).cpu().numpy()
            pred_arr[i*batch_size : i*batch_size + batch.size(0), :] = pred
            
    return pred_arr


def calculate_activation_statistics(dataloader, model, preprocess, device='cpu'):
    """
    Get mean vector and covariance matrix of the activation maps.
    """
    # Get activation maps
    act = get_activation(dataloader, model, preprocess, device)
    mu = np.mean(act, axis=0) # Mean
    sigma = np.cov(act, rowvar=False) # Covariance  
    
    return mu, sigma

from scipy import linalg
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    
    """
    Given Mean and Sigma of Real and Generated Data,
    it calculates FID between them using:
     
     d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
     
    """
    # Make sure they have appropriate dims
    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)
    
    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    
    # Handle various cases
    if not np.isfinite(covmean).all():
        msg = (
            "fid calculation produces singular product; "
            "adding %s to diagonal of cov estimates"
        ) % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError("Imaginary component {}".format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean

#### **Fréchet Inception Distance**

FID is a measure of similarity between two datasets of images. It was shown to correlate well with human judgement of visual quality and is most often used to evaluate the quality of samples of Generative Adversarial Networks. FID is calculated by computing the Fréchet distance between two Gaussians fitted to feature representations of the Inception network.

Given **Mean** and **Sigma** of Real and Generated Data,  
it calculates **FID** between them using:

**d² = ‖μ₁ - μ₂‖² + Tr(C₁ + C₂ - 2·√(C₁·C₂))**

Where:  
- **μ₁**, **C₁**: Mean and covariance of real data  
- **μ₂**, **C₂**: Mean and covariance of generated data  

Run the cell below to calculate **FID** ⬇️


In [5]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE' # could potentially produce incorrect numerical computations
# but there is no better workaround for now.

# loading config file
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config_path = r'config\default.yaml'

with open(config_path, 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

model_config = config['model_params']
train_config = config['train_params']
dataset_config = config['dataset_params']
diffusion_config = config['diffusion_params']

transform = transforms.Compose([transforms.Resize((model_config['im_size'],model_config['im_size'])),
                                transforms.Lambda(lambda x: x * 2 - 1)]) 
                                # Normalize to [-1, 1] ( similar to DDPM paper )

train_dataset = CustomDataset(dataset_config['im_path'], transform=transform)   
test_dataset = CustomDataset(dataset_config['test_im_path'], transform=transform)
generated_dataset = CustomDataset(os.path.join(train_config['task_name'],dataset_config['generated_im_path'])
                                  , transform=transform)

print( f' Train Dataset Size     : {len(train_dataset)}')   
print( f' Test Dataset Size      : {len(test_dataset)}')    
print( f' Generated Dataset Size : {len(generated_dataset)}')      


# Transform to Convert Output of CustomDataset class to Inception format.
import torchvision.transforms as transforms

transform_inception = transforms.Compose([
    transforms.Lambda(lambda x: (x + 1.0)/2.0), # [-1, 1] => [0, 1]
    transforms.ToPILImage(), # Tensor to PIL Image 
    transforms.Resize((299, 299)),
    transforms.Grayscale(num_output_channels=3),  # Convert to RGB format
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization

])

# Load InceptionV3 Model
import torchvision.models as models
# from torchvision.models.inception import Inception_V3_Weights
# model = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
model = models.inception_v3(pretrained=True) # ImageNet 1K weights loaded ( confirmed from documentation)
model.fc = nn.Identity()

# Mean and Sigma for Train Data
print( '\nCalculating Mean and Sigma for Train Data...')        
train_dataloader = DataLoader(train_dataset, batch_size = 64 , shuffle=False)
mu3, sigma3 = calculate_activation_statistics(train_dataloader, model, preprocess = transform_inception,
                                               device=device)

# Mean and Sigma for Test Data
print( '\nCalculating Mean and Sigma for Test Data...') 
test_dataloader = DataLoader(test_dataset, batch_size = 64  , shuffle=False)
mu2, sigma2 = calculate_activation_statistics(test_dataloader, model, preprocess = transform_inception,
                                               device=device)

# Mean and Sigma For Generated Data
print( '\nCalculating Mean and Sigma for Generated Data...')    
generated_dataloader = DataLoader(generated_dataset, batch_size= 64 , shuffle=False)
mu1, sigma1 = calculate_activation_statistics(generated_dataloader, model, preprocess = transform_inception, 
                                              device=device)


# Calculate FID between Generated and Train Data
fid = calculate_frechet_distance(mu1, sigma1, mu3, sigma3)
print(f'FID-Score (b/w train and generated data ): {fid}')  ## standard in literature.


# Calculate FID between Generated and Test Data
fid = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)
print(f'FID-Score (b/w test and generated data ): {fid}')




 Train Dataset Size     : 9000
 Test Dataset Size      : 1000
 Generated Dataset Size : 10000

Calculating Mean and Sigma for Train Data...


141it [00:26,  5.30it/s]



Calculating Mean and Sigma for Test Data...


16it [00:02,  5.83it/s]



Calculating Mean and Sigma for Generated Data...


157it [00:27,  5.74it/s]


FID-Score (b/w train and generated data ): 10.537900226665158
FID-Score (b/w test and generated data ): 11.65205665853604


**Note:** Similar to the original paper "Denoising Diffusion Probabilistic Models" by Ho et al. (2020). We have computed FID score between train-generated data ( standard practise in literature to use train set) and between test-generated data. Also, for reliable computation of FID computation we need atleast 5000-10000 images minimum ( empherical evidence).